In [1]:
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix

# Check GPU availability
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Set TensorFlow threading
tf.config.threading.set_inter_op_parallelism_threads(8)
tf.config.threading.set_intra_op_parallelism_threads(8)

# Set seed
random.seed(42)

# Dataset paths
dataset_path = "dataset/watermelon-disease/Augmented Image/Augmented_Image"
train_path = os.path.join(dataset_path, "train")
val_path = os.path.join(dataset_path, "val")

# Create train/val folders
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# Split raw images
classes = [d for d in os.listdir(dataset_path)
           if os.path.isdir(os.path.join(dataset_path, d)) and d not in ["train", "val"]]

for cls in classes:
    class_dir = os.path.join(dataset_path, cls)
    images = [img for img in os.listdir(class_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    random.shuffle(images)
    split_idx = int(len(images) * 0.8)
    train_images, val_images = images[:split_idx], images[split_idx:]

    os.makedirs(os.path.join(train_path, cls), exist_ok=True)
    os.makedirs(os.path.join(val_path, cls), exist_ok=True)

    for img in train_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(train_path, cls, img))
    for img in val_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(val_path, cls, img))

    if not os.listdir(class_dir):
        os.rmdir(class_dir)

print("✅ Dataset successfully split.")

# Get number of classes
num_classes = len(classes)

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)
test_set = test_datagen.flow_from_directory(
    val_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

print("Class labels:", training_set.class_indices)

# Build model using Functional API
input_tensor = Input(shape=(128, 128, 3))
base_model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output_tensor)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train
history = model.fit(
    training_set,
    steps_per_epoch=training_set.samples // training_set.batch_size,
    epochs=20,
    validation_data=test_set,
    validation_steps=test_set.samples // test_set.batch_size,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluation
predictions = model.predict(test_set, steps=test_set.samples // test_set.batch_size)
y_pred = np.argmax(predictions, axis=1)
y_true = test_set.classes[:len(y_pred)]

f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f"\n📊 Evaluation Metrics:")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Confusion matrix & classification report
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

report = classification_report(y_true, y_pred, target_names=list(training_set.class_indices.keys()))
print("Classification Report:")
print(report)

# Save Keras model
os.makedirs("model", exist_ok=True)
keras_model_path = "model/melon-disease.keras"
model.save(keras_model_path)
print(f"\n✅ Saved Keras model: {keras_model_path}")

# Export to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_model_path = "model/melon-disease.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"✅ Saved TFLite model: {tflite_model_path}")


2025-05-29 22:19:06.001809: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-29 22:19:06.010565: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-29 22:19:06.032805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748528346.069240  114191 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748528346.080590  114191 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-29 22:19:06.123820: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Num GPUs Available: 0


2025-05-29 22:19:11.428944: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Dataset successfully split.
Found 4620 images belonging to 4 classes.
Found 1155 images belonging to 4 classes.
Class labels: {'Anthracnose': 0, 'Downy_Mildew': 1, 'Healthy': 2, 'Mosaic_Virus': 3}


/tmp/ipykernel_114191/504692607.py:89: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')


...

144/144 ━━━━━━━━━━━━━━━━━━━━ 75s 525ms/step - accuracy: 0.9375 - loss: 0.3169 - val_accuracy: 0.9332 - val_loss: 0.1931 - learning_rate: 0.0010
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 383s 3s/step - accuracy: 0.8919 - loss: 0.2925 - val_accuracy: 0.9262 - val_loss: 0.1853 - learning_rate: 0.0010
Epoch 4/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 83s 575ms/step - accuracy: 0.9688 - loss: 0.1340 - val_accuracy: 0.9271 - val_loss: 0.1795 - learning_rate: 0.0010
Epoch 5/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 411s 3s/step - accuracy: 0.9110 - loss: 0.2340 - val_accuracy: 0.9219 - val_loss: 0.1899 - learning_rate: 0.0010
Epoch 6/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 84s 586ms/step - accuracy: 0.9375 - loss: 0.0851 - val_accuracy: 0.9236 - val_loss: 0.1853 - learning_rate: 0.0010
Epoch 7/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 383s 3s/step - accuracy: 0.9226 - loss: 0.2038 - val_accuracy: 0.9271 - val_loss: 0.1656 - learning_rate: 0.0010
Epoch 8/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 75s 523ms/step - accuracy: 0.8750 - loss: 0.295

INFO:tensorflow:Assets written to: /tmp/tmpn0_cu01q/assets


Saved artifact at '/tmp/tmpn0_cu01q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  140164486133744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164486177968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164486182720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164486175680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164486179904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164486178496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164446931072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164446933888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164446929312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140164446932832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14016448617462

W0000 00:00:1748531460.742073  114191 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1748531460.742097  114191 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-05-29 23:11:00.742462: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpn0_cu01q
2025-05-29 23:11:00.759102: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-05-29 23:11:00.759140: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpn0_cu01q
I0000 00:00:1748531460.916243  114191 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-05-29 23:11:00.942579: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-05-29 23:11:01.882839: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpn0_cu01q
2025-05-29 23:11:02.115326: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 

✅ Saved TFLite model: model/melon-disease.tflite
